In [2]:
import brightway2 as bw

Blank project:

In [3]:
bw.projects.set_current('uncertain cf test')

In [5]:
bw.bw2setup()

Creating default biosphere

Applying strategy: normalize_units

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00



Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds
Title: Writing activities to SQLite3 database:
  Started: 06/16/2017 13:21:28
  Finished: 06/16/2017 13:21:29
  Total time elapsed: 00:00:00
  CPU %: 63.300000
  Memory %: 1.284657
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.24 seconds
Wrote 718 LCIA methods with 178008 characterization factors
Creating core data migrations



Some random exchange:

In [6]:
some_exchange = bw.Database('biosphere3').random()
some_exchange, some_exchange['code']

('Scandium' (kilogram, None, ('soil',)),
 '8bd01869-7f83-4fcd-a167-af5735fa7ffd')

In [7]:
some_exchange.key

('biosphere3', '8bd01869-7f83-4fcd-a167-af5735fa7ffd')

CF for the random exchange, with uniform distribution:

In [18]:
my_cf = [(some_exchange.key,
          {"amount": 10,
           "uncertainty_type": 4,
           "minimum": 0,
           "maximum": 20}
         )]

In [19]:
uncertain_method = bw.Method(("fake", "method", "with uncertainty with _"))
my_metadata = {"unit": "fake unit", "why": "for testing uncertain CFs"}
uncertain_method.register(**my_metadata)

In [20]:
uncertain_method.write(my_cf)

LCI database:

In [11]:
simple_LCI_db = bw.Database('simple LCI db')
simple_LCI_db.register()

Activity with emission of "some_exchange"

In [12]:
simple_LCI_db.write(
    {('simple LCI db', 'some_code'): 
        {'name': 'fake activity',
         'unit': 'amount',
         'exchanges': 
            [
                {'input': ('simple LCI db', 'some_code'),
                 'amount': 1,
                 'type': 'production'},
                {'input': some_exchange.key,
                 'amount': 1,
                 'type': 'biosphere'},                
            ]
        },
    })

Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/16/2017 13:25:45
  Finished: 06/16/2017 13:25:45
  Total time elapsed: 00:00:00
  CPU %: 0.000000
  Memory %: 1.644879


Static LCA:

In [13]:
lca = bw.LCA({('simple LCI db', 'some_code'):1}, ('fake', 'method', 'with uncertainty'))

In [14]:
lca.lci()
lca.lcia()
lca.score

10.0

Monte Carlo - works:

In [21]:
mc = bw.MonteCarloLCA({('simple LCI db', 'some_code'):1}, ('fake', 'method', 'with uncertainty with _'))

In [22]:
next(mc)

10.0

In [23]:
next(mc)

10.0

Change CF uncertainty to beta - works:

In [16]:
my_cf = [(some_exchange.key,
          {"amount": 10,
           "uncertainty type": 10,
           "loc": 2,
           "scale": 50,
           "shape": 5}
         )]

In [17]:
uncertain_method.write(my_cf)

In [18]:
mc = bw.MonteCarloLCA({('simple LCI db', 'some_code'):1}, ('fake', 'method', 'with uncertainty'))

In [19]:
results = []
for _ in range(10):
    results.append(next(mc))
results

[14.083023483571155,
 5.293176751270895,
 22.120178247900764,
 18.52767191130432,
 13.5894067692715,
 9.118196687951025,
 17.412745816894923,
 17.37213396991942,
 7.463595923080009,
 12.05213903956038]